* Checkout new git branch for changes
* Overview of Implementing Delete logic
* Update user detail HTML page for Deletion
* Review logic to delete data using flask shell
* Add delete user logic to the application
* Review delete user logic
* Run application and validate
* Validate CI/CD Pipeline using GitHub Actions
* Exercise and Solution

* Checkout new git branch for changes
  * Use command `git checkout -b feature/cruddelete`

* Overview of Implementing Delete logic
  * `DELETE` is one of the REST APIs
  * HTML Forms typically supports `GET` and `POST`
  * We will implement the `DELETE` functionality using `GET`. It can also be implemented using `POST`.
  * We will add **Delete** button as part of user detail page. We also need to update the `user` function in `routes/users.py`.

* Update user detail HTML page for Deletion

Update `templates/user_detail.html`

```html
{%- extends "base.html" %}

{% block content %}
    <h3>User Details</h3>
    <div class="container">
        <form method="GET" action="{{ url_for('user', id=user.id) }}">
            <input type="text" name="id" value="{{ user.id }}" hidden /> 
            <label for="first_name"><b>First Name</b></label>
            {{ user.first_name }}
            <br><br>
            <label for="last_name"><b>Last Name</b></label>
            {{ user.last_name }}
            <br><br>
            <label for="username"><b>Username</b></label>
            {{ user.username }}
            <br><br>
            <label for="email"><b>Email</b></label>
            {{ user.email }}
            <br><br>

            <button type="submit" name="action" value="edit">Edit</button>
            <button type="submit" name="action" value="delete">Delete</button>
        </form>
    </div>
{%- endblock %}
```

* Review logic to delete data using flask shell

Launch flask shell using `python -m flask shell` and run the below code snippets.

```python
from models.user import User
User.query.all() # Use the first id
u = User.query.get(1)
db.session.delete(u)
db.session.commit() # Updates the data in the database table

User.query.all()
# The result will not contain the record deleted
```

Use pgAdmin and run the following query to confirm.
```sql
SELECT * FROM users WHERE id = 1;
```

* Add delete user logic to the application

Update `user` function in `app.py` with delete functionality.

```python
@app.route('/user', methods=['GET', 'POST'])
def user():
    if request.method == 'GET':
        id = request.args.get('id')
        if id:
            user = User.query.get(id)
            form_action = request.args.get('action')
            if form_action == 'edit':
                return render_template('user_form.html', user=user)
            elif form_action == 'delete':
                db.session.delete(user)
                db.session.commit()
                return redirect(url_for('users'))
            return render_template('user_detail.html', user=user)
        else:
            return render_template('user_form.html', user=None)
    elif request.method == 'POST':
        id = request.form['id']
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        username = request.form['username']
        email = request.form['email']
        if id:
            user = User.query.get(id)
            user.first_name = first_name
            user.last_name = last_name
            user.username = username
            user.email = email
        else:
            user = User(
                first_name=first_name, 
                last_name=last_name, 
                username=username,
                email=email
            )
            db.session.add(user)
        db.session.commit()
        return redirect(url_for('users'))
```

* Review delete user logic
  * Changes to user detail page
  * Changes to application to delete the data.

* Run application and validate

Run the application using Flask.
```shell
python -m flask run
```

Here are the steps to validate the application.
* Visit Users Dashboard http://localhost:5000/users
* Visit User Details page for 1 - http://localhost:5000/users?id=1
* Click on **Delete** button.
* The page will be redirected to users page. Make sure to review and confirm if the user is deleted from the database or not.
* You can also connect to pgAdmin and run `SELECT` to confirm if the user is deleted or not.

* Validate CI/CD Pipeline using GitHub Actions

1. Push changes to remote repository
2. Create Pull request to merge changes from feature branch to main branch
3. Validate GitHub Action or CI/CD Pipeline
4. Validate Application in GCP VM

* Exercise - Develop functionality to delete course details from the table.

1. Update course detail html page (course_form.html) with **Delete** button delete existing course details. 
2. Update function `course` for `/course` end point with `
GET` to delete data in the table.
3. Run and Validate the application. Go to the end point `/course` with `?course_id=1` parameter and perform below steps.
  * Make sure to click on **Delete** button.
  * Make sure that the page is redirected to **courses** dashboard.
  * The course deleted should not be there in the dashboard.
4. Make sure to cleanup the branch after validating the application in the remote VM

* Solution - Develop functionality to delete course details to the table.

Update `templates/course_detail.html` with logic to add delete button.

```html
{%- extends "base.html" %}

{% block content %}
    <h3>Course Details</h3>
    <div class="container">
        <form method="GET" action="{{ url_for('course', course_id=course.course_id) }}">
            <input type="text" name="course_id" value="{{ course.course_id }}" hidden /> 
            <label for="course_name"><b>Course Name</b></label>
            {{ course.course_name }}
            <br><br>
            <label for="course_author"><b>Course Author</b></label>
            {{ course.course_author }}
            <br><br>
            <label for="course_endpoint"><b>Course Endpoint</b></label>
            {{ course.course_endpoint }}
            <br><br>

            <button type="submit" name="action" value="edit">Edit</button>
            <button type="submit" name="action" value="delete">Delete</button>
        </form>
    </div>
{%- endblock %}
```

Update `course` function in `app.py` with relevant logic to delete when Delete button is pressed.

```python
@app.route('/course', methods=['GET', 'POST'])
def course():
    if request.method == 'GET':
        course_id = request.args.get('course_id')
        if course_id:
            course = Course.query.get(course_id)
            return render_template('course_detail.html', course=course)
        else:
            return render_template('course_form.html', course=None)
    elif request.method == 'POST':
        course_id = request.args.get('course_id')
        if course_id:
            course = Course.query.get(course_id)
            form_action = request.form['action']
            if form_action == 'edit':
                return render_template('course_form.html', course=course)
            elif form_action == 'delete':
                db.session.delete(course)
                db.session.commit()
                return redirect(url_for('courses'))
        else:
            course_id = request.form['course_id']
            course_name = request.form['course_name']
            course_author = request.form['course_author']
            course_endpoint = request.form['course_endpoint']
            if course_id:
                course = Course.query.get(course_id)
                course.course_name = course_name
                course.course_author = course_author
                course.course_endpoint = course_endpoint
            else:
                course = Course(
                    course_name=course_name, 
                    course_author=course_author, 
                    course_endpoint=course_endpoint
                )
                db.session.add(course)
            db.session.commit()
            return redirect(url_for('courses'))
```

Restart the application and then go to `http://localhost:5000/course?course_id=1`. Click on Delete, you will be redirected to `http://localhost:5000/courses`. Confirm whether course related to course_id 1 is deleted or not.